<a href="https://colab.research.google.com/github/gubosd/hello-world/blob/master/2023_Transformer_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Transformer Test

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

In [3]:
translator = pipeline("translation_en_to_fr")

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
s = translator("This is the transformer. Nice to meet you.")
print(s)

[{'translation_text': "C'est le transformateur et c'est bon de vous rencontrer."}]


In [31]:
model = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
import numpy as np
img = np.random.randn(100,100,3)

In [34]:
result = model(question='What is the his major?', context='He buy a linear algebra book.')
result

{'score': 0.9432556629180908,
 'start': 9,
 'end': 23,
 'answer': 'linear algebra'}

In [25]:
type(result[1]), len(result[1])

(dict, 2)

In [30]:
result

[{'score': 0.7667220830917358, 'label': 'kimono'},
 {'score': 0.12431895732879639, 'label': "pajama, pyjama, pj's, jammies"},
 {'score': 0.033992696553468704, 'label': 'gown'},
 {'score': 0.012362624518573284, 'label': 'overskirt'},
 {'score': 0.010159374214708805, 'label': 'hair slide'}]

### Bert 테스트
- https://stackoverflow.com/questions/69195950/problem-with-inputs-when-building-a-model-with-tfbertmodel-and-autotokenizer-fro

In [35]:
from transformers import TFBertModel, BertTokenizer

In [36]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [121]:
bert_input = bert_tokenizer.encode('good day', return_tensors='tf')
bert_input

<tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[ 101, 1363, 1285,  102]], dtype=int32)>

In [70]:
bert_input = bert_tokenizer.encode('mama how is it a good day', return_tensors='tf')
bert_input

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 12477,  1918,  1293,  1110,  1122,   170,  1363,  1285,
          102]], dtype=int32)>

In [112]:
ins = []
s = bert_tokenizer.encode_plus('good day', max_length=10, padding='max_length')
ins.append(s['input_ids'])
s = bert_tokenizer.encode_plus('good day sunshine', max_length=10, padding='max_length')
ins.append(s['input_ids'])
s = bert_tokenizer.encode_plus('nice day', max_length=10, padding='max_length')
ins.append(s['input_ids'])

ins

[[101, 1363, 1285, 102, 0, 0, 0, 0, 0, 0],
 [101, 1363, 1285, 18080, 102, 0, 0, 0, 0, 0],
 [101, 3505, 1285, 102, 0, 0, 0, 0, 0, 0]]

In [96]:
r = bert_model(np.array(ins, dtype='int32'))

In [97]:
for i in r:
  print(i)

last_hidden_state
pooler_output


In [98]:
r['pooler_output'].shape

TensorShape([3, 768])

In [99]:
r['last_hidden_state'].shape

TensorShape([3, 10, 768])

In [132]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input((10,), dtype='int32') # 'int32' is necessary
x = bert_model(inputs)
model = keras.models.Model(inputs=inputs, outputs=x)

for layer in model.layers[:2]:
  layer.trainable=False

In [118]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 10)]              0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             10, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [119]:
result = model.predict(ins)
result[0].shape, result[1].shape # pooler_output, last_hidden_state

1/1 [==============================] - 4s 4s/step


((3, 10, 768), (3, 768))

In [133]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input((10,), dtype='int32') # 'int32' is necessary
x = bert_model(inputs)[0]
x = keras.layers.Conv1D(8, 3, padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1)(x)
model = keras.models.Model(inputs=inputs, outputs=x)

for layer in model.layers[:2]:
  layer.trainable=False

model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 10)]              0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             10, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                          